In [1]:
from google.colab import files
uploaded = files.upload()   # upload your label_encoder_scikit.pkl


Saving xgb_model.pkl to xgb_model.pkl
Saving label_encoder_scikit.pkl to label_encoder_scikit.pkl
Saving cnn_final_model.keras to cnn_final_model.keras


In [2]:
from google.colab import files
uploaded = files.upload()

Saving scanner_hybrid_final.keras to scanner_hybrid_final.keras


In [24]:
# Install pyngrok if not installed
!pip install --quiet pyngrok

from pyngrok import conf, ngrok

token = input("Paste your ngrok authtoken (copied from dashboard.ngrok.com): ").strip()
conf.get_default().auth_token = token
ngrok.set_auth_token(token)   # sets token for current session
print("ngrok auth token set for this session.")


Paste your ngrok authtoken (copied from dashboard.ngrok.com): 36KbvsehmRjekDuNZQw3SZTwXul_7RzuwyZroHSPaQpSkdAyM
ngrok auth token set for this session.


In [25]:
# Make sure pyngrok and streamlit installed
!pip install --quiet pyngrok streamlit

from pyngrok import ngrok, conf
import time, os

# If you used Option A to set token, conf already has it.
# If you used Option B, set it here:
# conf.get_default().auth_token = os.environ["NGROK_AUTH_TOKEN"]

# Kill any previous tunnels just in case
ngrok.kill()

# Open a public HTTP tunnel on port 8501 (Streamlit default)
public_tunnel = ngrok.connect(addr=8501, proto="http")
print("Public URL (open this in a new browser tab):", public_tunnel.public_url)

# Run Streamlit in background (app.py must exist in current directory)
# Use nohup & so it keeps running while the cell completes
cmd = "nohup streamlit run app.py --server.port 8501 --server.address 0.0.0.0 > streamlit.log 2>&1 &"
os.system(cmd)
time.sleep(2)
print("Streamlit started in background. If the page doesn't load, check streamlit.log for errors.")

Public URL (open this in a new browser tab): https://autopsic-unperipherally-ngoc.ngrok-free.dev
Streamlit started in background. If the page doesn't load, check streamlit.log for errors.


In [3]:
!pip install --quiet scipy


In [23]:
%%writefile app.py
import streamlit as st
import numpy as np
import tensorflow as tf
from PIL import Image, ImageOps, ImageFilter
import pickle, joblib, io, os
import matplotlib.pyplot as plt

# for stats
try:
    from scipy.stats import skew, kurtosis, entropy as sp_entropy
except Exception:
    skew = None
    kurtosis = None
    sp_entropy = None

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# -------------------------
# App settings
# -------------------------
st.set_page_config(page_title="ScannerID", layout="centered")
st.title("📠 ScannerID — Scanner Identification System")

IMG_SIZE = (256, 256)

# -------------------------
# Load models
# -------------------------
@st.cache_resource
def load_models():
    cnn = tf.keras.models.load_model("cnn_final_model.keras")
    with open("xgb_model.pkl", "rb") as f:
        xgb = pickle.load(f)
    with open("label_encoder_scikit.pkl", "rb") as f:
        le = pickle.load(f)
    return cnn, xgb, le

cnn_model, xgb_model, label_encoder = load_models()
classes = list(label_encoder.classes_)

# -------------------------
# Preprocessing helpers
# -------------------------
def preprocess_for_cnn(img: Image.Image):
    """Return array shape (1,256,256,1) for CNN"""
    imgg = ImageOps.grayscale(img)
    imgg = imgg.filter(ImageFilter.MedianFilter(3))
    imgg = imgg.resize(IMG_SIZE)
    arr = np.array(imgg).astype("float32") / 255.0
    arr = arr.reshape(1, IMG_SIZE[0], IMG_SIZE[1], 1)
    return arr

def compute_image_features(img: Image.Image):
    """
    Compute an 8-d feature vector:
    [mean, std, median, min, max, skewness, kurtosis, entropy]
    """
    imgg = ImageOps.grayscale(img)
    imgg = imgg.resize(IMG_SIZE)
    arr = np.array(imgg).astype("float32").ravel() / 255.0  # flatten 0-1

    # basic stats
    mean_v = float(np.mean(arr))
    std_v = float(np.std(arr))
    median_v = float(np.median(arr))
    min_v = float(np.min(arr))
    max_v = float(np.max(arr))

    # skew, kurtosis (fall back to manual if scipy unavailable)
    if skew is not None:
        try:
            skew_v = float(skew(arr))
        except Exception:
            skew_v = 0.0
    else:
        # manual skew: mean((x-mean)^3)/std^3
        if std_v > 0:
            skew_v = float(np.mean(((arr - mean_v) ** 3))) / (std_v ** 3 + 1e-12)
        else:
            skew_v = 0.0

    if kurtosis is not None:
        try:
            kurt_v = float(kurtosis(arr, fisher=False))
        except Exception:
            kurt_v = 0.0
    else:
        # manual kurtosis (population): mean((x-mean)^4)/std^4
        if std_v > 0:
            kurt_v = float(np.mean(((arr - mean_v) ** 4))) / (std_v ** 4 + 1e-12)
        else:
            kurt_v = 0.0

    # entropy using histogram
    try:
        hist, _ = np.histogram(arr, bins=256, range=(0.0, 1.0), density=True)
        # small eps to avoid log(0)
        hist = hist + 1e-12
        if sp_entropy is not None:
            ent_v = float(sp_entropy(hist))
        else:
            ent_v = float(-np.sum(hist * np.log(hist)))
    except Exception:
        ent_v = 0.0

    feat = np.array([mean_v, std_v, median_v, min_v, max_v, skew_v, kurt_v, ent_v], dtype="float32")
    return feat.reshape(1, -1)  # shape (1,8)

# -------------------------
# Prediction helpers
# -------------------------
def predict_cnn(arr):
    probs = cnn_model.predict(arr)[0]
    return probs

def predict_xgb(feat_arr):
    # xgb_model may expect different input type (sklearn wrapper); pass as-is
    pred = xgb_model.predict(feat_arr)
    # if predict returns class indices or class labels
    if isinstance(pred, np.ndarray):
        p = pred[0]
    else:
        p = pred
    # try to inverse transform if xgb returns numeric class index
    try:
        return label_encoder.inverse_transform([int(p)])[0]
    except Exception:
        return str(p)

# -------------------------
# Sidebar
# -------------------------
st.sidebar.header("Model Status")
st.sidebar.write(f"CNN input shape: {cnn_model.inputs[0].shape}")
# Try to show XGBoost expected features if present
xgb_nf = getattr(xgb_model, "n_features_in_", None)
if xgb_nf is not None:
    st.sidebar.write(f"XGBoost expects {xgb_nf} features")
else:
    st.sidebar.write("XGBoost feature count: unknown")

st.sidebar.success("Models loaded")

mode = st.sidebar.selectbox("Prediction mode", ["CNN Only", "XGBoost Only", "Ensemble (CNN+XGB)"])

st.write("Upload image (png/jpg/jpeg/tif/tiff). The app will preprocess to 256×256 grayscale for CNN, and compute an 8-d feature vector for XGBoost.")

# -------------------------
# File upload & prediction
# -------------------------
uploaded = st.file_uploader("Upload an image", type=["png","jpg","jpeg","tif","tiff"])
if uploaded:
    img = Image.open(io.BytesIO(uploaded.read()))
    st.image(img, caption="Uploaded image", use_column_width=True)

    # preprocess for cnn
    cnn_arr = preprocess_for_cnn(img)

    # compute 8-d features for xgb
    xgb_feat = compute_image_features(img)

    # If XGBoost expects not 8 but some other number, warn user:
    if xgb_nf is not None and xgb_nf != xgb_feat.shape[1]:
        st.warning(f"Warning — XGBoost expects {xgb_nf} features but computed {xgb_feat.shape[1]} features. Predictions may be invalid. If you have the original feature extraction code, provide it and I will integrate it.")

    # Do predictions
    if mode == "CNN Only":
        probs = predict_cnn(cnn_arr)
        label = classes[np.argmax(probs)]
        conf = float(np.max(probs))
        st.write(f"**CNN prediction:** {label} (conf {conf:.3f})")

    elif mode == "XGBoost Only":
        xb_lab = predict_xgb(xgb_feat)
        st.write(f"**XGBoost prediction:** {xb_lab}")

    else:
        probs = predict_cnn(cnn_arr)
        cnn_label = classes[np.argmax(probs)]
        cnn_conf = float(np.max(probs))
        xb_lab = predict_xgb(xgb_feat)
        # simple ensemble logic
        if cnn_label == xb_lab:
            final = cnn_label
        else:
            final = cnn_label if cnn_conf >= 0.6 else xb_lab
        st.write(f"**Ensemble prediction:** {final}")
        st.write(f"- CNN: {cnn_label} ({cnn_conf:.3f})")
        st.write(f"- XGBoost: {xb_lab}")

# -------------------------
# Confusion matrix ad-hoc logger
# -------------------------
st.markdown("---")
st.header("Confusion Matrix Logger (ad-hoc)")
if "true" not in st.session_state:
    st.session_state.true = []
    st.session_state.pred = []

if uploaded:
    true_label = st.selectbox("Select true class for this image", classes)
    if st.button("Add to evaluation data"):
        # use last prediction (prefer CNN)
        try:
            last_pred = final  # may have been set in ensemble branch
        except NameError:
            try:
                # fallback to cnn prediction
                last_pred = classes[np.argmax(predict_cnn(cnn_arr))]
            except Exception:
                last_pred = None
        if last_pred is not None:
            st.session_state.true.append(true_label)
            st.session_state.pred.append(last_pred)
            st.success("Saved sample to evaluation buffer")
        else:
            st.warning("No prediction available to log")

if len(st.session_state.true) > 1:
    cm = confusion_matrix(st.session_state.true, st.session_state.pred, labels=classes)
    fig, ax = plt.subplots(figsize=(8,6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(ax=ax, xticks_rotation=45, cmap=plt.cm.Blues)
    st.pyplot(fig)


Overwriting app.py
